### 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助你：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(agent101)`，即可执行下方代码

In [28]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate agent101

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "agent101" ]; then
    echo "✅ 当前单元格已成功激活到 agent101 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(agent101)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate agent101"
fi

echo "=========================================" 

== Conda 环境检查报告 (仅针对当前 Bash 子进��) ==
✅ 当前单元格已成功激活到 agent101 环��。
✅ 正在使用的环境路径: /root/miniconda3/envs/agent101

💡 提示: 后续的Python��元格将使用Notebook当前选择的Jupyter内核。
   如果需要后续单元格也使用此��境，请执行以下操作:
   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。


In [29]:
# 2. 设置pip 为清华源
%pip config list -v set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list -v list


For variant 'global', will try loading '/etc/xdg/pip/pip.conf'
For variant 'global', will try loading '/etc/pip.conf'
For variant 'user', will try loading '/root/.pip/pip.conf'
For variant 'user', will try loading '/root/.config/pip/pip.conf'
For variant 'site', will try loading '/root/miniconda3/envs/agent101/pip.conf'
ERROR: Got unexpected number of arguments, expected 0. (example: "/root/miniconda3/envs/agent101/bin/python -m pip config list")
Note: you may need to restart the kernel to use updated packages.
For variant 'global', will try loading '/etc/xdg/pip/pip.conf'
For variant 'global', will try loading '/etc/pip.conf'
For variant 'user', will try loading '/root/.pip/pip.conf'
For variant 'user', will try loading '/root/.config/pip/pip.conf'
For variant 'site', will try loading '/root/miniconda3/envs/agent101/pip.conf'
ERROR: Got unexpected number of arguments, expected 0. (example: "/root/miniconda3/envs/agent101/bin/python -m pip config list")
Note: you may need to restart th

In [30]:
# 3. 设置HuggingFaceAgent
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT

env: HF_ENDPOINT=https://hf-mirror.com
https://hf-mirror.com


In [31]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                               |
|:-------------|:-----------------------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                                           |
| CPU 信息     | 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz (1 physical cores, 4 logical cores) |
| 内存信息     | 5.75 GB (Available: 1.55 GB)                                                       |
| GPU 信息     | No GPU found (nvidia-smi not found)                                                |
| CUDA 信息    | CUDA not found                                                                     |
| Python 版本  | 3.10.18                                                                            |
| Conda 版本   | conda 24.4.0                                         

<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Example - Trace and Evaluate LangGraph Agents" description: "This guide shows how to evaluate LangGraph Agents with Langfuse using online and offline evaluation methods." category: "Integrations" -->

# LangGraph Agent追踪与评估完整指南

## 📖 教程概述

在本教程中，我们将深入学习如何使用 [Langfuse](https://langfuse.com)（一个强大的大模型可观测性平台）与 [Hugging Face Datasets](https://huggingface.co/datasets)，来**全面监控 [LangGraph Agent](https://github.com/langchain-ai/langgraph) 的执行过程（traces）**并**科学评估其性能表现**。

## 🎯 学习目标

本指南将帮助你掌握将 AI Agent快速且可靠地部署到生产环境所需的核心技能：
- **在线评估**：实时监控生产环境中的Agent表现
- **离线评估**：使用基准数据集进行系统性测试


## 🔍 为什么 AI Agent评估如此重要？

在 AI Agent开发过程中，评估是确保系统质量的关键环节：

- **🐛 问题诊断**：当Agent任务执行失败或结果不理想时，能够快速定位问题根源
- **📊 性能监控**：实时追踪系统的成本消耗、响应延迟等关键指标
- **🔄 持续改进**：通过用户反馈和评估数据，不断提升Agent的可靠性与安全性
- **🚀 生产就绪**：确保Agent在真实环境中能够稳定运行


## 🛠️ 步骤 0：环境准备与依赖安装

### 📦 安装核心依赖库

在开始本教程之前，我们需要安装以下核心库：

- **`langgraph`**：用于构建多节点、状态驱动的 AI Agent工作流
- **`langfuse`**：提供大模型应用的可观测性和评估功能  
- **`langchain`** 系列：用于 LLM 应用开发的核心框架
- **`datasets`**：Hugging Face 的数据集处理库


<!-- CALLOUT_START type: "info" emoji: "⚠️" -->
**📌 重要提示：**
- 本教程使用 **Langfuse Python SDK v3**，它提供了更好的性能和新特性
- 建议在虚拟环境中运行本教程以避免依赖冲突
<!-- CALLOUT_END -->


In [32]:
# 📦 安装所需的Python包
# 使用魔法命令 %pip 在Jupyter环境中安装依赖库
%pip install langfuse==3.3.0 langchain==0.3.27 langgraph==0.6.7 langchain-openai==0.3.31 langchain_community==0.3.27 langchain_huggingface==0.3.1

# 各库功能说明：
# - langfuse: LLM应用的可观测性和评估平台
# - langchain: 大语言模型应用开发框架
# - langgraph: 基于langchain的图形化工作流构建工具
# - langchain_openai: OpenAI模型的langchain集成
# - langchain_community: 社区贡献的langchain扩展
# - langchain_huggingface: Hugging Face模型的langchain集成

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


## 🔑 1：配置 API 密钥和环境变量

### 获取 Langfuse API 密钥

在开始使用 Langfuse 之前，你需要获取 API 访问凭证：

#### 方案一：使用 Langfuse Cloud（推荐）
1. 访问 [Langfuse Cloud](https://cloud.langfuse.com) 并注册账户
2. 创建新项目或选择现有项目
3. 在项目设置页面获取以下密钥：
   - `LANGFUSE_PUBLIC_KEY`：以 `pk-lf-` 开头的公钥
   - `LANGFUSE_SECRET_KEY`：以 `sk-lf-` 开头的私钥

#### 方案二：自托管 Langfuse
如果你选择自托管部署，请按照 [Langfuse 自托管文档](https://langfuse.com/docs/deployment/self-host) 进行配置。

### 获取 OpenAI API 密钥

1. 访问 [OpenAI 平台](https://platform.openai.com/)
2. 注册账户并完成身份验证
3. 在 API 密钥页面创建新的 API 密钥
4. 确保账户有足够的余额用于 API 调用

### 🔐 安全提醒

- **请勿将 API 密钥硬编码在代码中**
- **生产环境建议使用环境变量或密钥管理系统**
- **定期轮换密钥以提高安全性**

In [ ]:
# 🔐 环境变量配置 - 安全存储敏感信息
# 环境变量是存储API密钥等敏感信息的优秀做法
# 避免在代码中硬编码密钥，防止泄露

import os, getpass

def _set_env(var: str):
    """
    安全地设置环境变量
    如果环境变量不存在，会提示用户输入
    使用getpass模块隐藏输入内容，防止密码泄露
    """
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# 🤖 OpenAI API 配置
# OpenAI API密钥：从 https://platform.openai.com/api-keys 获取
# 这是调用GPT模型必需的认证信息
_set_env("OPENAI_API_KEY")

# APIAgent地址：如果你使用第三方Agent服务（如国内Agent）
# 示例：https://api.apiyi.com/v1
# 如果直接使用OpenAI官方API，可以留空
_set_env("OPENAI_BASE_URL")

# 🌐 Langfuse 配置
# Langfuse是一个可观测性平台，需要注册账户获取密钥
# 注册地址：https://cloud.langfuse.com

# 公开密钥：用于标识你的项目
_set_env("LANGFUSE_PUBLIC_KEY")

# 秘密密钥：用于认证，请妥善保管
_set_env("LANGFUSE_SECRET_KEY")

# 服务器地址：选择离你最近的区域
# 🇪🇺 欧盟区域(推荐) https://cloud.langfuse.com
# 🇺🇸 美国区域 https://us.cloud.langfuse.com
_set_env("LANGFUSE_HOST")

# 💡 初学者提示：
# 1. 环境变量存储在操作系统中，重启后需要重新设置
# 2. 生产环境中建议使用.env文件或云服务配置
# 3. 永远不要在代码中硬编码API密钥！

### 🔗 连接验证与客户端初始化

设置完环境变量后，我们需要初始化 Langfuse 客户端并验证连接。

**核心概念解释：**
- **`get_client()`**：Langfuse 提供的便捷函数，会自动读取环境变量中的凭证
- **客户端实例**：用于与 Langfuse 服务器通信的对象
- **连接验证**：确保 API 密钥正确且网络连接正常

In [34]:
# 📡 导入 Langfuse 客户端并建立连接
from langfuse import get_client

# 🔧 初始化 Langfuse 客户端
# get_client() 会自动从环境变量中读取 API 凭证
langfuse = get_client()

# ✅ 验证 API 连接和身份认证
# auth_check() 方法会测试与 Langfuse 服务器的连接
if langfuse.auth_check():
    print("✅ Langfuse 客户端连接成功！API 认证通过")
    print("🎯 现在可以开始追踪和评估 LLM 应用了")
else:
    print("❌ 认证失败！请检查以下项目：")
    print("   1. API 密钥是否正确设置")
    print("   2. 服务器地址是否正确")
    print("   3. 网络连接是否正常")

✅ Langfuse 客户端连接成功！API 认证通过
🎯 现在可以开始追踪和评估 LLM 应用了


## 🔬 2：构建并观测复杂的邮件处理Agent

### 🎯 进阶实战：真实业务场景模拟

现在我们来构建一个更加复杂且贴近实际业务场景的Agent系统。

### 📧 业务场景：智能邮件管理助手

我们将创建一个**邮件处理Agent**，具备以下功能：

#### 🔧 核心功能模块
- **📬 邮件接收**：读取和解析邮件内容
- **🔍 垃圾邮件识别**：智能判断邮件是否为垃圾邮件
- **🗂️ 自动分类**：对合法邮件进行分类处理
- **✍️ 回复起草**：为重要邮件生成回复草稿
- **📢 通知主人**：向韦恩先生汇报重要邮件

#### 📊 追踪的高级指标

通过这个复杂Agent，我们将观察以下关键指标：
- **💰 成本追踪**：详细的令牌消耗和 API 费用
- **⏱️ 性能分析**：每个处理步骤的耗时分布
- **🔄 工作流路径**：Agent的决策逻辑和执行路径
- **❌ 错误监控**：异常情况的捕获和分析

### 🏗️ 技术架构特点

- **状态驱动**：使用 LangGraph 的状态管理机制
- **条件分支**：根据邮件类型执行不同的处理逻辑
- **多节点协作**：模拟真实的业务处理流程

In [37]:
# 📦 导入构建复杂Agent所需的库

import os  # 操作系统接口，用于环境变量管理
from typing import TypedDict, List, Dict, Any, Optional  # 类型注解，提高代码可读性和IDE支持
from langgraph.graph import StateGraph, START, END  # LangGraph核心组件：状态图、开始节点、结束节点
from langchain_openai import ChatOpenAI  # OpenAI模型的LangChain集成
from langchain_core.messages import HumanMessage  # LangChain消息类型

print("📚 库导入完成，准备构建邮件处理Agent...")
print("🔧 即将使用的核心组件：")
print("   - StateGraph: 构建状态驱动的工作流")
print("   - ChatOpenAI: 调用 GPT 模型进行智能处理")
print("   - TypedDict: 定义严格的数据结构")

📚 库导入完成，准备构建邮件处理Agent...
🔧 即将使用的核心组件：
   - StateGraph: 构建状态驱动的工作流
   - ChatOpenAI: 调用 GPT 模型进行智能处理
   - TypedDict: 定义严格的数据结构


In [38]:
# 🏗️ 定义邮件处理Agent的状态结构

class EmailState(TypedDict):
    """
    邮件处理Agent的状态数据结构

    这个类定义了Agent在处理邮件过程中需要维护的所有状态信息
    """
    # 📧 原始邮件信息
    email: Dict[str, Any]  # 包含发件人、主题、正文等邮件完整信息

    # 🔍 垃圾邮件检测结果
    is_spam: Optional[bool]  # 是否为垃圾邮件（True/False/None）

    # ✍️ 回复草稿
    draft_response: Optional[str]  # 为主人准备的回复草稿

    # 💬 对话历史记录
    messages: List[Dict[str, Any]]  # 存储处理过程中的LLM对话记录

print("✅ 邮件状态结构定义完成")
print("📋 状态字段说明：")
print("   - email: 原始邮件数据")
print("   - is_spam: 垃圾邮件判定结果")
print("   - draft_response: 回复草稿")
print("   - messages: LLM对话历史")

✅ 邮件状态结构定义完成
📋 状态字段说明：
   - email: 原始邮件数据
   - is_spam: 垃圾邮件判定结果
   - draft_response: 回复草稿
   - messages: LLM对话历史


In [39]:
# ✅ 初始化大语言模型（LLM），后续所有节点都会复用它进行推理
model = ChatOpenAI(model="gpt-4o", temperature=0)

# 🔁 定义工作流中的每个节点函数
def read_email(state: EmailState):
    """
    入口节点：展示邮件基础信息，帮助我们在命令行中观察流程。
    """
    email = state["email"]  # 从状态中取出当前邮件
    print(f"阿尔弗雷德正在处理来自 {email['sender']} 的邮件，主题为：{email['subject']}")
    return {}  # 节点只做展示，不修改状态

def classify_email(state: EmailState):
    """
    使用 LLM 判断当前邮件是否为垃圾邮件。
    如果是垃圾邮件就不记录模型对话，避免污染历史。
    """
    email = state["email"]

    # 构造提示词，向 LLM 传入邮件的所有关键信息（中文初学者友好版本）
    prompt = f"""
请以阿尔弗雷德（Alfred，韦恩先生的管家，同时知晓其“蝙蝠侠”身份）的视角，分析下面这封邮件，判断其是垃圾邮件（SPAM）还是正常邮件（HAM），并说明是否需要提醒韦恩先生注意。

邮件内容：
发件人（From）：{email['sender']}
主题（Subject）：{email['subject']}
正文（Body）：{email['body']}

请先判断这封邮件是否为垃圾邮件。
只返回一个英文单词作为最终答案：若是垃圾邮件，返回“SPAM”；若是正常邮件，返回“HAM”。不要输出多余文字。
答案：
    """
    messages = [HumanMessage(content=prompt)]  # LangChain 要求传入 HumanMessage 对象
    response = model.invoke(messages)  # 调用 LLM 获得判定结果

    response_text = response.content.lower()  # 统一转小写，便于关键词匹配
    print(response_text)  # 在控制台输出，方便我们调试和观察
    is_spam = "spam" in response_text and "ham" not in response_text  # 同时排除同时出现 spam/ham 的情况

    if not is_spam:
        # 如果不是垃圾邮件，就将本次问答追加到对话历史中，供后续节点使用
        new_messages = state.get("messages", []) + [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response.content}
        ]
    else:
        # 垃圾邮件无需记录上下文，保持原有的消息记录
        new_messages = state.get("messages", [])

    return {
        "is_spam": is_spam,       # 把垃圾邮件判定结果写回状态
        "messages": new_messages  # 同步对话历史
    }

def handle_spam(state: EmailState):
    """
    垃圾邮件分支：这里只演示打印提示语，真实项目可以写入数据库或报警。
    """
    print("阿尔弗雷德已经将邮件标记为垃圾邮件。")
    print("该电子邮件已被移至垃圾邮件文件夹。")
    return {}  # 返回空字典表示不修改状态字段

def drafting_response(state: EmailState):
    """
    合法邮件分支：让 LLM 帮忙撰写一份礼貌的回复草稿。
    """
    email = state["email"]

    # 维持提示词，明确输出语气和需要覆盖的关键内容（中文初学者友好版本）
    prompt = f"""
请以阿尔弗雷德（Alfred，韦恩先生的管家）的口吻，为下面这封邮件起草一份礼貌、简洁且专业的初稿回复。

邮件内容：
发件人（From）：{email['sender']}
主题（Subject）：{email['subject']}
正文（Body）：{email['body']}

请生成一段简短、专业、语气友善的中文回复草稿，供韦恩先生审阅并在发送前个性化润色。
    """

    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    # 将最新的问答追加到对话历史里，保持上下文完整
    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]

    return {
        "draft_response": response.content,  # 保存生成的邮件草稿
        "messages": new_messages
    }

def notify_mr_wayne(state: EmailState):
    """
    收尾节点：模拟向布鲁斯·韦恩汇报邮件处理结果。
    """
    email = state["email"]

    print("" + "="*50)
    print(f"Sir, you've received an email from {email['sender']}.")
    print(f"Subject: {email['subject']}")
    print("I've prepared a draft response for your review:")
    print("-"*50)
    print(state["draft_response"])
    print("="*50 + "")

    return {}

# 🧭 路由逻辑：根据垃圾邮件判定选择下一步的分支
def route_email(state: EmailState) -> str:
    if state["is_spam"]:
        return "spam"
    else:
        return "legitimate"

# 🛠️ 创建状态图，将上面定义的节点串联成一个 LangGraph 工作流
email_graph = StateGraph(EmailState)

# 📌 注册节点——每一行都会把函数变成图里的一个执行节点
email_graph.add_node("read_email", read_email)  # 首先读取并展示邮件信息
email_graph.add_node("classify_email", classify_email)  # 然后请 LLM 判定垃圾邮件
email_graph.add_node("handle_spam", handle_spam)  # 垃圾邮件走单独的处理分支
email_graph.add_node("drafting_response", drafting_response)  # 合法邮件生成回复草稿
email_graph.add_node("notify_mr_wayne", notify_mr_wayne)  # 最后向主人汇报结果


In [40]:
# ➕ 配置节点之间的流转顺序
email_graph.add_edge(START, "read_email")  # 图的起点先进入 read_email 节点

# 🧠 判定之后根据结果流向不同分支
email_graph.add_edge("read_email", "classify_email")  # 展示完邮件后调用分类逻辑

# 🔀 添加条件分支：route_email 返回字符串决定下一条边
email_graph.add_conditional_edges(
    "classify_email",  # 根据垃圾邮件判定结果来决定去向
    route_email,
    {
        "spam": "handle_spam",          # 判定为垃圾邮件则直接走 handle_spam 节点
        "legitimate": "drafting_response"  # 合法邮件则继续撰写回复
    }
)

# ✅ 收尾：无论哪个分支走完都回到 END 节点
email_graph.add_edge("handle_spam", END)  # 垃圾邮件处理完毕即结束
email_graph.add_edge("drafting_response", "notify_mr_wayne")  # 回复草稿后通知主人
email_graph.add_edge("notify_mr_wayne", END)  # 汇报结束后整个流程收尾


In [41]:
# 🧮 将图结构编译成可执行的 LangGraph Agent对象
compiled_graph = email_graph.compile()


In [42]:
# 📨 准备两封示例邮件，帮助我们观察不同分支的执行效果
legitimate_email = {
    "sender": "京东客服",  # 发件人
    "subject": "关于你近期订单的发票开具说明",  # 邮件主题
    "body": "尊敬的韦恩先生，你好！关于你在京东的近期订单，增值税电子普通发票已开具并推送至你的邮箱。如需纸质发票或抬头变更，请在7日内通过“我的订单-申请开票”发起，我们将尽快处理。给你带来不便，敬请谅解。"  # 邮件正文
}

spam_email = {
    "sender": "某数字货币项目方",  # 垃圾邮件常见的推销者
    "subject": "限时暴涨100倍，立即上车！",  # 诱导性标题
    "body": "韦恩先生，我们新上线了一款数字货币，承诺稳稳赚、稳赚不赔！扫码加群，前100名赠送空投名额，错过今天再等一年！"  # 明显的垃圾推广/诈骗文案
}


In [45]:
from langfuse import observe, get_client
from langfuse.langchain import CallbackHandler

@observe()
def process_email_with_langgraph(email_data):
    langfuse = get_client()
    
    # 设置 trace name
    langfuse.update_current_trace(
        name="email-processing-workflow",  # 自定义 trace 名称
        input={"email": email_data}
    )
    
    # 初始化 CallbackHandler
    langfuse_handler = CallbackHandler()
    
    # 执行 LangGraph
    result = compiled_graph.invoke(
        input={
            "email": email_data,
            "is_spam": None,
            "draft_response": None,
            "messages": []
        },
        config={"callbacks": [langfuse_handler]}
    )
    
    # 更新 trace 输出
    langfuse.update_current_trace(output=result)
    
    return result

# ✅ 运行合法邮件示例，演示完整工作流
result = process_email_with_langgraph(legitimate_email)
# 🚨 再运行垃圾邮件示例，观察分支差异
result = process_email_with_langgraph(spam_email)

阿尔弗雷德正在处理来自 京东客服 的邮件，主题为：关于你近期订单的发票开具说明


'NoneType' object has no attribute 'items'
Traceback (most recent call last):
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 751, in on_llm_end
    llm_usage = _parse_usage(response)
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 1027, in _parse_usage
    llm_usage = _parse_usage_model(response.llm_output[key])
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 946, in _parse_usage_model
    for key, value in input_token_details.items():
AttributeError: 'NoneType' object has no attribute 'items'


ham


'NoneType' object has no attribute 'items'
Traceback (most recent call last):
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 751, in on_llm_end
    llm_usage = _parse_usage(response)
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 1027, in _parse_usage
    llm_usage = _parse_usage_model(response.llm_output[key])
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 946, in _parse_usage_model
    for key, value in input_token_details.items():
AttributeError: 'NoneType' object has no attribute 'items'


Sir, you've received an email from 京东客服.
Subject: 关于你近期订单的发票开具说明
I've prepared a draft response for your review:
--------------------------------------------------
尊敬的京东客服团队：

您好！感谢您及时通知我关于近期订单发票的相关信息。我已收到电子发票，若有进一步需求，我会在规定时间内通过“我的订单-申请开票”功能提交申请。

感谢您提供的优质服务，祝工作顺利！

此致  
敬礼  
韦恩
阿尔弗雷德正在处理来自 某数字货币项目方 的邮件，主题为：限时暴涨100倍，立即上车！


'NoneType' object has no attribute 'items'
Traceback (most recent call last):
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 751, in on_llm_end
    llm_usage = _parse_usage(response)
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 1027, in _parse_usage
    llm_usage = _parse_usage_model(response.llm_output[key])
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 946, in _parse_usage_model
    for key, value in input_token_details.items():
AttributeError: 'NoneType' object has no attribute 'items'


spam
阿尔弗雷德已经将邮件标记为垃圾邮件。
该电子邮件已被移至垃圾邮件文件夹。


##### 备注
```
'NoneType' object has no attribute 'items'
Traceback (most recent call last):
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 751, in on_llm_end
    llm_usage = _parse_usage(response)
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 1027, in _parse_usage
    llm_usage = _parse_usage_model(response.llm_output[key])
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 946, in _parse_usage_model
    for key, value in input_token_details.items():
AttributeError: 'NoneType' object has no attribute 'items'
```
这个错误是 Langfuse LangChain 回调处理器在解析 LLM 响应的 token 使用详情时遇到的问题。虽然错误不影响程序运行(邮件处理流程正常完成),但会在日志中产生干扰信息。
这里可能和我使用OpenAI国内代理API有关系

### 追踪结构

Langfuse 会记录包含若干 **span（跨度）** 的**trace（追踪）**，每个 span 代表Agent逻辑中的一个步骤。本例中的追踪包含整体运行以及如下子跨度：
- 工具调用（
- LLM 调用（使用 'gpt-4o' 的 Responses API）

你可以检查这些记录以精确了解时间消耗、令牌使用量等：

![Langfuse 中的追踪树](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509251730026.png)

_[前往该追踪](https://cloud.langfuse.com/project/cmequpe0j00euad07w6wrvkzg/traces?peek=2d1f23b960fb1ff0bdaf7623fda4936c&timestamp=2025-09-25T09%3A06%3A06.476Z)_

## 🔬 3：在线评估

在线评估指在真实线上环境（生产环境的实际使用中）对Agent进行评估。这需要对真实用户交互进行持续监控与结果分析。

我们在此总结了多种评估技术的指南：[链接](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)。

### 生产环境常见监控指标

1. **成本（Costs）**：埋点会记录令牌用量，你可按每个令牌的价格估算成本。
2. **延迟（Latency）**：观察完成每个步骤或整次运行所需的时间。
3. **用户反馈（User Feedback）**：用户可直接提供反馈（如点赞/点踩）以帮助迭代与修正Agent。
4. **LLM 评审（LLM-as-a-Judge）**：使用额外的 LLM 近实时评估Agent输出（如检测毒性或正确性）。

下面展示这些指标的示例。

#### 1. 成本（Costs）

下图展示了 `gpt-4o` 调用的用量，可据此识别高成本步骤并优化Agent。

![成本](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509251732570.png)

_[前往该追踪](https://cloud.langfuse.com/project/cmequpe0j00euad07w6wrvkzg/traces?peek=2d1f23b960fb1ff0bdaf7623fda4936c&timestamp=2025-09-25T09%3A06%3A06.476Z)_

#### 2. 延迟（Latency）

还可以查看完成每个步骤所需的时间。如下例所示，整个运行约 3 秒，你可以细分到各步骤。此举有助于识别瓶颈并优化Agent。

![延迟](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509251735069.png)

_[前往该追踪](https://cloud.langfuse.com/project/cmequpe0j00euad07w6wrvkzg/traces?peek=2d1f23b960fb1ff0bdaf7623fda4936c&timestamp=2025-09-25T09%3A06%3A06.476Z&display=timeline)_

#### 3. 用户反馈（User Feedback）

如果你的Agent嵌入在用户界面中，可以采集用户的直接反馈（例如在聊天界面中的点赞/点踩）。

用户反馈通常借助：**为追踪添加评分** 方式

[评分（Score）](https://langfuse.com/docs/scores/overview) 用于评价单个观测（observation）或整条追踪（trace），可帮助你在运行时执行自定义质量检查，或配合人工审核流程。

下面的示例演示如何：

- 为某个 span 记录一个数值型评分（如 `relevance`）
- 为整条追踪记录一个分类型评分（如 `feedback`）

这有助于系统化地评估与改进应用质量。

**→ 想深入了解？请参阅 [Langfuse 自定义评分指南](https://langfuse.com/docs/scores/custom)。**

In [52]:
from langfuse import get_client
# ═══════════════════════════════════════════════════════════════════════════
# 📊 Langfuse 评分系统详解
# ═══════════════════════════════════════════════════════════════════════════
# 
# 评分（Score）是一种对 Trace（追踪）进行质量评估和反馈的机制
# 
# 💡 评分的主要应用场景：
# ┌─────────────────────────────────────────────────────────────────────────┐
# │ 1. 【在线实时反馈】                                                        │
# │    - 用户满意度评分（👍/👎）                                              │
# │    - 答案是否有帮助（helpful/not helpful）                                │
# │    - 用户点击"好评"或"差评"按钮的即时反馈                                   │
# │                                                                          │
# │ 2. 【离线质量度量】                                                        │
# │    - AI 回答的准确性（accuracy）评分                                       │
# │    - 内容相关性（relevance）评分                                          │
# │    - 安全性检测（safety check）评分                                       │
# │    - 事实核查（fact-checking）结果                                        │
# │                                                                          │
# │ 3. 【A/B 测试与对比实验】                                                  │
# │    - 不同模型版本的效果对比                                                │
# │    - 不同提示词（prompt）的性能评估                                        │
# │    - 多种策略的横向比较                                                    │
# └─────────────────────────────────────────────────────────────────────────┘
#
# ═══════════════════════════════════════════════════════════════════════════
langfuse = get_client()

# ═══════════════════════════════════════════════════════════════════════════
# 🎯 方案一：使用 span 对象的 score_trace() 方法（★ 推荐方案，最直观）
# ═══════════════════════════════════════════════════════════════════════════
#
# 📌 核心特点：
#    - 在上下文管理器（with）内部使用
#    - 直接通过 span 对象调用 score_trace() 方法
#    - 代码清晰，作用域明确，最容易理解和维护
#
# ✅ 适用场景：
#    1. 同步执行流程中，需要立即对当前 trace 进行评分
#    2. 评分逻辑就在 LangGraph 执行代码附近
#    3. 需要在一个明确的代码块内完成评分（推荐用于大多数场景）
#
# 💡 使用时机：
#    - LangGraph 执行完成后，立即获取结果并评分
#    - 在同一个函数或代码块中处理执行和评分逻辑
#    - 适合同步场景，评分发生在业务逻辑执行的同一上下文中
#
# 🔧 技术细节：
#    - span 是上下文管理器返回的对象，代表当前的追踪范围
#    - score_trace() 会自动关联到当前 span 所属的 trace
#    - 无需手动管理 trace_id，框架自动处理关联关系
with langfuse.start_as_current_span(name="email-processing-workflow") as span:
    # ... 此处执行 LangGraph 的业务逻辑 ...
    # 例如：调用 LLM、执行工具函数、处理用户输入等
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 📝 直接通过 span 对象记录评分
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    span.score_trace(
        name="user-feedback-test-01",        # 评分项名称（可自定义，如：accuracy、relevance、safety）
        value=1,                            # 评分值（NUMERIC 类型时为数字；TEXT/CATEGORICAL 时为字符串）
        data_type="NUMERIC",                # 数据类型：NUMERIC（数值）/ TEXT（文本）/ CATEGORICAL（分类）
        comment="This was correct, thank you"  # 可选：评分备注或用户评论
    )
    # 
    # 📊 参数说明：
    # ├─ name: 评分维度的名称标识（如：user-feedback、accuracy、relevance）
    # ├─ value: 评分的具体值
    # │         • NUMERIC: 可以是整数或小数（如：1, 0.95, 5.0）
    # │         • TEXT: 文本评论（如："答案准确且详细"）
    # │         • CATEGORICAL: 分类标签（如："excellent"、"good"、"poor"）
    # ├─ data_type: 评分值的数据类型，影响 Langfuse 平台的展示和聚合方式
    # └─ comment: 附加说明，便于后续分析时理解评分背景

In [53]:
# ═══════════════════════════════════════════════════════════════════════════
# 🎯 方案二：使用全局方法 score_current_trace()（适用于深层函数调用）
# ═══════════════════════════════════════════════════════════════════════════
#
# 📌 核心特点：
#    - 仍然在 with 上下文内部，但不需要直接持有 span 对象
#    - 通过 langfuse.score_current_trace() 自动找到当前活跃的 trace
#    - 适合在深层嵌套的函数中调用，无需层层传递 span 参数
#
# ✅ 适用场景：
#    1. 在深层函数中进行评分，不方便传递 span 对象
#    2. 回调函数或钩子函数中需要评分
#    3. 代码结构中 span 对象不在当前作用域，但仍在同一个上下文中
#
# 💡 使用时机：
#    - 评分逻辑在子函数、工具函数或回调中实现
#    - 不想污染函数签名（不想添加 span 参数）
#    - 多层函数调用，但都在同一个 trace 上下文中
#
# 🔧 技术细节：
#    - 依赖 Langfuse 的上下文管理机制（类似 threading.local）
#    - 自动检测当前执行栈中最近的活跃 trace
#    - 必须在 with 语句的作用域内调用，否则会找不到当前 trace
#
# 🆚 与方案一的区别：
#    ┌──────────────┬─────────────────────┬─────────────────────┐
#    │              │ 方案一 (span.score) │ 方案二 (score_current)│
#    ├──────────────┼─────────────────────┼─────────────────────┤
#    │ span 对象    │ 必须持有            │ 不需要持有          │
#    │ 代码位置     │ 通常在 with 块内部  │ 可以在深层函数中    │
#    │ 参数传递     │ 需要传递 span       │ 无需传递参数        │
#    │ 代码耦合度   │ 低（显式引用）      │ 更低（隐式查找）    │
#    │ 适用场景     │ 简单直接的评分      │ 复杂嵌套的评分      │
#    └──────────────┴─────────────────────┴─────────────────────┘
#
with langfuse.start_as_current_span(name="email-processing-workflow") as span:
    # ... 此处执行 LangGraph 的业务逻辑 ...
    
    # 假设在某个深层函数中需要评分，但 span 对象不方便传递
    # 这时可以直接调用 langfuse.score_current_trace()
    langfuse.score_current_trace(
        name="user-feedback-test-02",
        value=1,
        data_type="NUMERIC"
    )
    # 
    # 🔍 内部机制：
    # Langfuse 维护了一个线程局部存储（thread-local storage），记录当前活跃的 trace
    # score_current_trace() 会查找当前线程中最近激活的 trace 并进行评分
    # 这种方式避免了手动传递 span 对象，代码更简洁
    #


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# 🎯 方案三：使用 trace_id 直接创建评分（★ 适合异步/后台批处理）
# ═══════════════════════════════════════════════════════════════════════════
#
# 📌 核心特点：
#    - 完全脱离上下文管理器（with 语句）
#    - 需要明确提供 trace_id 来关联评分
#    - 最灵活，支持跨请求、跨服务、跨时间的评分
#
# ✅ 适用场景：
#    1. 异步评分：LangGraph 执行完成后，用户稍后才给出反馈
#    2. 批量评分：从数据库读取历史 trace_id，批量添加评分
#    3. 离线分析：定期运行脚本对历史数据进行质量评估
#    4. 跨服务评分：前端收集反馈后，通过 API 提交到后端进行评分
#    5. 人工标注：人工审核团队对历史对话进行打分
#
# 💡 使用时机：
#    - 评分与执行不在同一时间（时间解耦）
#    - 评分与执行不在同一进程（空间解耦）
#    - 需要对历史数据进行回溯评分
#    - 实现用户反馈系统（用户点击"有帮助"按钮时）
#
# 🔧 技术细节：
#    - trace_id 必须提前保存（可通过 Langfuse.create_trace_id() 生成或从回调中获取）
#    - 完全独立的 API 调用，不依赖任何上下文
#    - 可以在任何时间、任何地方调用，只要有正确的 trace_id
#
# 🆚 与前两种方案的区别：
#    ┌─────────────┬────────────┬────────────┬────────────────┐
#    │             │ 方案一      │ 方案二      │ 方案三         │
#    ├─────────────┼────────────┼────────────┼────────────────┤
#    │ 上下文依赖  │ 需要        │ 需要        │ 不需要         │
#    │ trace_id    │ 自动获取    │ 自动获取    │ 手动提供       │
#    │ 时间耦合    │ 同步        │ 同步        │ 完全解耦       │
#    │ 执行时机    │ 执行期间    │ 执行期间    │ 任何时候       │
#    │ 典型场景    │ 即时评分    │ 深层评分    │ 异步/批量评分  │
#    │ 灵活性      │ 低          │ 中          │ 高             │
#    │ 复杂度      │ 低          │ 低          │ 中（需管理ID） │
#    └─────────────┴────────────┴────────────┴────────────────┘
#
# 📝 典型使用流程：
# 1️⃣ 在 LangGraph 执行时保存 trace_id
# 2️⃣ 将 trace_id 与业务数据关联（如：存入数据库）
# 3️⃣ 在合适的时机（如用户反馈、定时任务）使用 trace_id 进行评分
#
langfuse.create_score(
    trace_id="f4122db2c96eb4752585c3706cf802f2",  # ⚠️ 这里需要替换成真实的 trace_id
    name="user-feedback",                   # 评分项名称
    value=1,                                # 评分值
    data_type="NUMERIC",                    # 数据类型
    comment="This was correct, thank you"   # 可选：评分说明
)
# 
# 🌟 实际应用示例：
# 
# 【场景 1：用户延迟反馈】
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1. 用户提问 → LangGraph 执行 → 返回答案（保存 trace_id 到数据库）
# 2. 用户阅读答案后点击"👍 有帮助"按钮（可能是几分钟后）
# 3. 前端发送 trace_id 到后端 API
# 4. 后端调用 create_score() 记录用户反馈
#
# 【场景 2：批量质量评估】
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1. 每天定时任务从数据库读取昨天的所有 trace_id
# 2. 对每个 trace 运行自动化评估脚本（如：事实核查、安全检测）
# 3. 使用 create_score() 批量添加评分结果
#
# 【场景 3：人工标注】
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1. 标注团队使用内部工具查看历史对话
# 2. 人工评估每个对话的质量（准确性、相关性、友好性等）
# 3. 标注工具调用 create_score() 将评分写入 Langfuse
#

# ═══════════════════════════════════════════════════════════════════════════
# 💡 优秀做法建议
# ═══════════════════════════════════════════════════════════════════════════
#
# 1. 【默认选择】大多数情况下使用方案一（span.score_trace）
#    ├─ 代码清晰，易于理解和维护
#    └─ 适合 80% 的同步评分场景
#
# 2. 【深层调用】需要在子函数中评分时使用方案二（score_current_trace）
#    ├─ 避免层层传递 span 对象
#    └─ 保持函数签名简洁
#
# 3. 【异步场景】需要延迟评分或批量处理时使用方案三（create_score）
#    ├─ 实现用户反馈系统
#    ├─ 定期批量评估
#    └─ 跨服务评分
#
# 4. 【组合使用】可以同时使用多种方案
#    ├─ 方案一：即时评分（如：AI 自动评估）
#    └─ 方案三：延迟评分（如：用户反馈）
#
# 5. 【数据类型选择】
#    ├─ NUMERIC: 用于可量化的指标（准确率、满意度分数）
#    ├─ TEXT: 用于文本评论和详细反馈
#    └─ CATEGORICAL: 用于分类标签（excellent/good/poor）
#
# ═══════════════════════════════════════════════════════════════════════════

用户反馈随后会被 Langfuse 捕获：

![Langfuse 中捕获的用户反馈](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511251751419.png)

#### 4. 自动化的 LLM 评审打分（LLM-as-a-Judge）

LLM-as-a-Judge 提供了一种自动评估Agent输出的方法。你可以**配置一个独立的 LLM 调用**，用于评估输出的正确性、毒性、风格或其他你关心的指标。

**工作流程：**
1. 定义一个**评估模板**，例如“检查文本是否含有毒性”。
2. 指定用于评审的模型（judge-model），例如 `gpt-4o-mini`。
2. 每当Agent生成输出时，将其与模板一起传给“评审”LLM。
3. 评审 LLM 给出评分或标签，并将结果记录到可观测性平台。

Langfuse 示例：

![LLM 评审模板](https://langfuse.com/images/cookbook/integration_openai-agents/evaluator-template.png)
![LLM 评审器](https://langfuse.com/images/cookbook/integration_openai-agents/evaluator.png)

#### 5. 可观测性指标总览

所有上述指标都可以在统一的仪表盘中可视化。这样你可以快速查看Agent在多次会话中的表现，并随时间跟踪质量指标。

![可观测性指标总览](https://langfuse.com/images/cookbook/integration_openai-agents/dashboard-dark.png)

## 🔬 4： 离线评估（Offline Evaluation）

在线评估可用于获取实时反馈，但同样需要进行**离线评估（offline evaluation）**——即在开发前或开发过程中进行系统性的检查。这样可以在发布变更到生产环境之前，保障质量与可靠性。

### 数据集评估（Dataset Evaluation）

在离线评估中，通常会：
1. 准备一个基准数据集（包含提示词与期望输出的成对样本）
2. 使用该数据集批量运行你的 Agent
3. 将模型输出与期望结果进行比较，或采用额外的自动打分机制

下面我们使用一个问答数据集示例：[q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset)，其中包含问题与期望答案。

In [18]:
%pip install datasets==4.3.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
from datasets import load_dataset

# 设置HuggingFaceAgent
%env HF_ENDPOINT=https://hf-mirror.com

# 📥 从 Hugging Face 下载示例数据集，这里包含问答形式的条目
dataset = load_dataset("junzhang1207/search-dataset", split="train")
df = pd.DataFrame(dataset)  # 转成 DataFrame 方便筛选与遍历
print("First few rows of search-dataset:")
print(df.head())


/root/miniconda3/envs/agent101/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


env: HF_ENDPOINT=https://hf-mirror.com
First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  Abou

In [20]:
import pandas as pd
import os

# 📌 替换为你本地文件的实际路径，例如：'dataset/data.jsonl '
local_jsonl_path = '../dataset/data.jsonl'

if os.path.exists(local_jsonl_path):
    try:
        # 使用 pd.read_json 并指定 lines=True 来读取 JSON Lines 格式
        local_df = pd.read_json(local_jsonl_path, lines=True)
        
        print(f"✅ 成功读取本地 JSONL 文件: {local_jsonl_path}")
        print("\n数据集前5行:")
        print(local_df.head())
        print(f"\n数据集形状: {local_df.shape}")
        
    except Exception as e:
        print(f"❌ 读取文件时发生错误: {e}")
else:
    print(f"文件不存在，请检查路径是否正确: {local_jsonl_path}")

✅ 成功读取本地 JSONL 文件: ../dataset/data.jsonl

数据集前5行:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers 

接下来，我们在 Langfuse 中创建一个数据集实体以追踪运行；随后将数据集中的每条记录添加到系统中。

In [21]:
from langfuse import Langfuse
langfuse = Langfuse()

langfuse_dataset_name = "qa-dataset_langgraph-agent"

# 🗂️ 在 Langfuse 中创建一个新的数据集，用于存储评测样本
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="从Hugging Face上传的问答数据集",
    metadata={
        "date": "2025-11-04",
        "type": "benchmark"
    }
)


Dataset(id='cmicvzreq000spl07o8xsvpcq', name='qa-dataset_langgraph-agent', description='从Hugging Face上传的问答数据集', metadata={'date': '2025-11-04', 'type': 'benchmark'}, project_id='cmicm34hk0006pl07fhwb5ezk', created_at=datetime.datetime(2025, 11, 24, 8, 30, 22, 842000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 11, 25, 2, 52, 43, 779000, tzinfo=datetime.timezone.utc), inputSchema=None, expectedOutputSchema=None)

In [22]:
# 🎯 仅选取 30 条示例数据上传，实际项目可根据需求调整
df_30 = local_df.sample(30)

for idx, row in df_30.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},            # Langfuse 需要明确的输入字段
        expected_output={"text": row["expected_answer"]}  # 提供标准答案便于后续评估
    )


![Langfuse 中的数据集条目](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509261143566.png)

#### 在数据集上运行Agent

首先，构建一个使用 OpenAI 模型回答问题的简易 LangGraph Agent。

In [23]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage  # 如需自定义输入消息可以使用该类型
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

# 🧱 定义状态结构：messages 字段会自动累积对话历史
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 🏗️ 初始化一个新的状态图构建器
graph_builder = StateGraph(State)

# 🤖 准备要调用的 OpenAI 聊天模型
llm = ChatOpenAI(model="gpt-4.5-preview")

def chatbot(state: State):
    """
    单节点聊天机器人：
    - 将当前所有消息传给 LLM
    - 返回模型的回复，LangGraph 会自动把它追加到状态里
    """
    return {"messages": [llm.invoke(state["messages"])]}

# 🔗 注册节点与入口、出口
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")

# ⚙️ compile() 会返回可直接调用的图实例
graph = graph_builder.compile()

接着，我们定义一个辅助函数 `my_agent()`，其职责是：
1. 创建一个 Langfuse 追踪（trace）
2. 获取 `langfuse_handler_trace`，用于为 LangGraph 的执行过程打点
3. 运行我们的 Agent，并在调用时传入 `langfuse_handler_trace` 以记录执行细节

In [24]:
from typing import Annotated

from langfuse import get_client
from langfuse.langchain import CallbackHandler
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate  # 可用于自定义提示模板（本示例暂未使用）
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

# 🏗️ 构建一个带 Langfuse 追踪能力的 LangGraph Agent
graph_builder = StateGraph(State)
llm = ChatOpenAI(model="gpt-4o")  # 选择对话模型  ；使用其他模型测试，注意修改
langfuse = get_client()  # 复用前面配置好的 Langfuse 客户端

def chatbot(state: State):
    """
    核心节点：将对话历史交给 LLM，并把生成结果包装成 LangGraph 需要的格式。
    """
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

def my_agent(question, langfuse_handler):
    """
    对外暴露的便捷函数：
    1. 打开一个 Langfuse span 以便观测这次请求；
    2. 调用 LangGraph Agent获取回答；
    3. 将输入输出写回 Langfuse，方便后续评估。
    """

    # 创建一个顶层追踪 span，所有上下文都会记录在这里
    with langfuse.start_as_current_span(name="my-langgraph-agent") as root_span:

        # Step 2: LangChain processing
        response = graph.invoke(
            input={"messages": [HumanMessage(content=question)]},
            config={"callbacks": [langfuse_handler]}
        )

        # 将原始问题和模型回答同步到 Langfuse 仪表盘
        root_span.update_trace(
            input=question,
            output=response["messages"][1].content)

        print(question)
        print(response["messages"][1].content)

    return response["messages"][1].content


最后，我们遍历数据集中的每一条样本，运行Agent，并将生成的追踪与该数据集条目进行关联。如有需要，还可以附加一个快速的评估分数。

### 测试： 使用gpt-4o模型

In [25]:
from langfuse import get_client
from langfuse.langchain import CallbackHandler

# 📡 初始化追踪组件：CallbackHandler 会把 LangChain 的每一步同步到 Langfuse
langfuse_handler = CallbackHandler()
langfuse = get_client()

dataset = langfuse.get_dataset('qa-dataset_langgraph-agent')  # 获取上一步创建的数据集

for item in dataset.items:
    # ✅ item.run() 会为每个样本开启一个子追踪，方便查看单条样本的执行情况
    with item.run(
        run_name="run_gpt-4o", # 使用其他模型测试，注意修改
        run_description="My first run",  # 使用其他模型测试，注意修改
        run_metadata={"model": "gpt-4o"},  # 使用其他模型测试，注意修改
    ) as root_span:
        # 进入此上下文的所有调用都会自动关联到当前 dataset item

        # 🎯 运行核心业务逻辑时，再开一个 generation 上下文记录单次模型调用
        with langfuse.start_as_current_generation(
            name="llm-call",
            model="gpt-4o",  # 使用其他模型测试，注意修改
            input=item.input
        ) as generation:
            # 用我们刚才封装的 my_agent 完成实际问答
            output = my_agent(str(item.input), langfuse_handler)
            generation.update(output=output)

        # 📝 可选择对结果打分（例如人工点评或自动指标）
        root_span.score_trace(
            name="user-feedback",
            value=1,
            comment="This is a comment",  # 可记录评分原因，便于回溯
        )

# 🔚 所有调用结束后刷新客户端，确保缓冲区里的数据都被发送
langfuse.flush()


'NoneType' object has no attribute 'items'
Traceback (most recent call last):
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 751, in on_llm_end
    llm_usage = _parse_usage(response)
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 1027, in _parse_usage
    llm_usage = _parse_usage_model(response.llm_output[key])
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 946, in _parse_usage_model
    for key, value in input_token_details.items():
AttributeError: 'NoneType' object has no attribute 'items'


{'text': 'How much did Microsoft pay to acquire Mojang, the creator of Minecraft?'}
Microsoft paid **$2.5 billion** to acquire Mojang, the developer of Minecraft, in September 2014.


'NoneType' object has no attribute 'items'
Traceback (most recent call last):
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 751, in on_llm_end
    llm_usage = _parse_usage(response)
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 1027, in _parse_usage
    llm_usage = _parse_usage_model(response.llm_output[key])
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 946, in _parse_usage_model
    for key, value in input_token_details.items():
AttributeError: 'NoneType' object has no attribute 'items'


{'text': 'Which video game is set in 1899 and 1907?'}
The video game set in 1899 and 1907 is *Red Dead Redemption 2*. Developed by Rockstar Games, it is an action-adventure game that follows the story of Arthur Morgan, a member of the Van der Linde gang, as they navigate challenges during the decline of the Wild West era.


'NoneType' object has no attribute 'items'
Traceback (most recent call last):
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 751, in on_llm_end
    llm_usage = _parse_usage(response)
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 1027, in _parse_usage
    llm_usage = _parse_usage_model(response.llm_output[key])
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 946, in _parse_usage_model
    for key, value in input_token_details.items():
AttributeError: 'NoneType' object has no attribute 'items'


...

{'text': 'how much co2 does photosynthesis produce annually'}
Photosynthesis does not produce carbon dioxide (CO2); instead, it is a process where plants, algae, and some bacteria *consume* CO2 and convert it into glucose (stored energy) and oxygen. During photosynthesis, organisms remove CO2 from the atmosphere and release oxygen as a byproduct.

Globally, photosynthesis plays a critical role in carbon cycling and removing vast amounts of CO2 annually. It's estimated that terrestrial and oceanic photosynthetic organisms absorb approximately **100–120 billion metric tons** of CO2 from the atmosphere each year. This natural process helps regulate atmospheric CO2 levels and supports ecosystems by providing energy for living organisms.

If you're asking how much **oxygen** photosynthesis produces annually, it's roughly proportional to the CO2 uptake, as oxygen is released during the conversion of CO2 and water into glucose. Let me know if you'd like clarification!


'NoneType' object has no attribute 'items'
Traceback (most recent call last):
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 751, in on_llm_end
    llm_usage = _parse_usage(response)
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 1027, in _parse_usage
    llm_usage = _parse_usage_model(response.llm_output[key])
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 946, in _parse_usage_model
    for key, value in input_token_details.items():
AttributeError: 'NoneType' object has no attribute 'items'


{'text': 'How many F1 world titles has Max Verstappen won?'}
As of October 2023, Max Verstappen has won **three Formula 1 World Championships**. He secured his titles in **2021**, **2022**, and **2023**, solidifying his dominance in the sport during these years.


'NoneType' object has no attribute 'items'
Traceback (most recent call last):
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 751, in on_llm_end
    llm_usage = _parse_usage(response)
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 1027, in _parse_usage
    llm_usage = _parse_usage_model(response.llm_output[key])
  File "/root/miniconda3/envs/agent101/lib/python3.10/site-packages/langfuse/langchain/CallbackHandler.py", line 946, in _parse_usage_model
    for key, value in input_token_details.items():
AttributeError: 'NoneType' object has no attribute 'items'


{'text': 'Which village in Donetsk Oblast was captured by Russian forces on 2 August 2024?'}
I currently do not have information on events or developments beyond October 2023. For the latest updates regarding conflicts or specific incidents in August 2024, I recommend consulting reliable news sources or official reports.


你可以在不同的 Agent 配置之间重复这一流程，例如：
- 模型（如 gpt-4o-mini、o1 等）
- 提示词（Prompts）
- 工具（如是否启用搜索能力）
- Agent 复杂度（多Agent vs 单Agent）

随后可在 Langfuse 中进行并排对比。在此示例中，我们在 30 条数据集问题上分别运行了 3 次Agent，每次使用不同的 OpenAI 模型。可以看到，随着模型能力增大，正确回答的数量按预期提升。

`Answer Correctness` 分数由一个[“模型充当评审”（LLM-as-a-Judge）评估器](https://langfuse.com/docs/scores/model-based-evals)生成，它会基于数据集中给出的参考答案来评估输出是否正确。

![数据集运行概览](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511041805188.png)
![数据集运行对比](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511041805669.png)


## 🔬 5： LLM-as-a-Judge 评估

### 一、创建新的 LLM-as-a-Judge 评估器

1. **导航**：进入 Langfuse 控制台`左侧栏`的 **Evaluators** 的` LLM-as-a-Judge `。
2. **操作**：点击 `Create Evaluator` 按钮以初始化一个新的评估配置。

![image-20251126165332628](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261653010.png)

### 二、设置默认模型(Set up default model)

您需要定义用于执行评估任务的默认 LLM（裁判模型）。

![image-20251126165906049](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261659568.png)

- `LLM adapter`:  `openai`

- `API Base URL`：`https://api.apiyi.com/v1`

- `API Key`：在`https://api.apiyi.com`申请

  > OpenAI国内代理地址
  > https://api.apiyi.com/register/?aff_code=we80
  > 新用户注册送0.1美金

设置后，选择模型名称

![image-20251126170023999](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261700395.png)

### 三、选择评估器 (Pick an Evaluator)

在此步骤中，您需要确定评估逻辑的来源，主要有两种方式，我们选择`托管评估器`：

1. **托管评估器 (Managed Evaluator)**：

   ![image-20251126170134380](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261701760.png)

   - **来源**：Langfuse 官方或合作伙伴（如 Ragas）维护的现成目录。
   - **内容**：涵盖常见的质量维度，如*幻觉 (Hallucination)*、*上下文相关性 (Context-Relevance)*、*毒性 (Toxicity)*、*有用性 (Helpfulness)*。
   - **优势**：无需编写 Prompt，即开即用，且持续更新。

   > 我们选择托管评估器中`Answer Correctness`评估器

2. **自定义评估器 (Custom Evaluator)**：

   - **适用场景**：当通用库无法满足特定的业务逻辑需求时。
   - **配置步骤**：
     1. 编写包含 `{{variables}}` 占位符的评估 Prompt（例如 `{{input}}`, `{{output}}`, `{{ground_truth}}`）。
     2. （可选）自定义评分标准（0-1分）和推理引导词，以规范 LLM 的输出。
     3. （可选）为此评估器绑定特定的模型（如果不绑定，则使用默认模型）。
   - **复用性**：保存后，该评估器可在整个项目中复用。

### 四、选择评估数据源

> 我们选择托管评估器中`Answer Correctness`评估器后，进入评估期设置

配置评估器运行的目标数据范围，分为**生产数据**和**数据集数据**，我们选择`数据集运行 (Dataset runs)`：：

1. **实时生产数据 (Live Data)**：

   - **监控目标**：实时监控线上应用的表现。
   - **范围 (Scope)**：建议选择 `New traces only`（仅新产生的 Trace）；也可选择回填历史数据。
   - **过滤器 (Filter)**：通过 Trace name、Tags、`userId` 等字段精确筛选。*强烈建议先通过 Filter 缩小范围*。
   - **预览**：系统会展示过去 24 小时内匹配当前 Filter 的 Trace 样本，供您核对。
   - **采样 (Sampling)**：配置运行百分比（例如 5%）以控制成本和评估吞吐量。

   ![image-20251126171503359](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261715850.png)

2. **数据集运行 (Dataset runs)**：

   - **过滤器 (Filter)**：通过 Datase字段精确筛选。*强烈建议先通过 Filter 缩小范围*。

   ![image-20251126172840308](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261728654.png)

### 五、映射变量并预览 Prompt

> 此步骤将 Trace 或数据集中的实际数据字段绑定到 Prompt 变量（如 `{{input}}`）。
>
> 1. **数据映射**：
>    - **Live Data**：通常将 Trace Input 映射到 `{{input}}`，Trace Output 映射到 `{{output}}`。
>    - **JSONPath 支持**：如果数据嵌套在 JSON 对象中，使用 JSONPath 表达式提取（例如 `$.choices[0].message.content`）。
>    - **Dataset**：系统通常会自动建议映射（如 Dataset 的 `expected_output` 映射到 `{{ground_truth}}`）。
> 2. **实时预览 (Live Preview)**：
>    - Langfuse 会使用匹配 Filter 的历史 Trace 填充 Prompt。
>    - **操作**：请务必切换几条不同的 Trace，检查 Prompt 中的数据填充是否符合预期，确保评估上下文完整。

![image-20251126172944653](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261730128.png)

##### 1. 确定评估范围 (Target Filter)

- **选择数据集**：在 "Target filter" 中，选择你要评估的特定数据集，如 `qa-dataset_langgraph-agent`。
- 这决定了评估器将针对哪些历史数据或测试集运行。

##### 2. 配置评估 Prompt 与变量映射 (Variable Mapping)

目的是告诉评估模型（Judge LLM）从哪里获取数据来填入 Prompt。

- **配置 `{{ground_truth}}`（基准事实）：**
  - **来源对象 (Object)**：选择 `Dataset item`（数据集条目）。
  - **来源变量 (Variable)**：选择 `Expected output`（预期输出/标准答案）。
  - *含义*：告诉评估器，Prompt 中的“基准事实”字段，应直接读取数据集中预设好的标准答案。
- **配置 `{{answer}}`（大模型回答）：**
  - **来源对象 (Object)**：选择 `Trace`（链路追踪数据）。
  - **来源变量 (Variable)**：选择 `Output`（模型实际输出）。
  - *含义*：告诉评估器，Prompt 中的“回答”字段，应读取大模型在实际运行（Trace）中生成的最终回复。

##### 3. 执行评估 (Execute)

- 点击右下角的 **"Execute"** 按钮。
- 系统将开始遍历数据集，将每一条的 `Expected output` 和对应的 Trace `Output` 填入你编写的 Prompt 中，调用大模型（Judge）进行 TP/FP/FN 的分类和打分。

### 六、查看评估过程

LLM-as-a-Judge 的每一次运行本身也是一条 Trace，您可以通过以下四种方式查看其详细执行过程（包括发送给裁判的 Prompt、Token 消耗、延迟等）：

![image-20251126175814773](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261758011.png)

- 在左侧导航栏中，点击 **"LLM-as-a-Judge"** 菜单项。
- **操作**：在 "Logs"（日志）一列下，点击 **"View"** 按钮

### 七、理解执行状态

监控评估器的运行状况：

- **Completed**：评估成功完成。
- **Error**：评估失败（点击 Execution trace ID 查看具体报错）。
- **Delayed**：触发了 LLM 提供商的速率限制，系统正在进行指数退避重试。
- **Pending**：评估任务已进入队列，等待运行。

![image-20251126175931708](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261759085.png)